## Submit spike sorting
This script generates cluster jobs for spikesorting (IBL pipeline)

In [1]:
import labdatatools as ldt
from pathlib import Path
import numpy as np
import subprocess

SUBJECT = 'MM012' #TODO: 9,10,11,12,13,15
N_PROBES = 3 # This is actually n shanks because IBL breaks shanks into individual binary files
GPU = 'RTX2080Ti'
CLUSTER = 'Hoffman' #Hoffman or Hodgkin, the cluster to submit to

In [2]:
files = ldt.rclone_list_files(SUBJECT).dropna()
ephys_sessions = files[files.dirname.str.contains('raw_ephys_data')]
ksfolders = files[files.dirname.str.contains('spike_sorters/kilosort2.5/') & files.filename.str.contains('spike_times.npy')]
ephys_sessions = np.unique([f'{s.session}/{s.datatype}' for i,s in ephys_sessions.iterrows()])

In [3]:
sorted_sessions, unsorted_sessions = [],[]
for sess in ephys_sessions:
    fold = ksfolders.dirname.str.contains(sess)
    if sum(fold) == N_PROBES:
        sorted_sessions.append(sess)
    else:
        unsorted_sessions.append(sess)

print(f'There are {len(sorted_sessions)} sorted sessions')
print(f'There are {len(unsorted_sessions)} unsorted sessions')

There are 42 sorted sessions
There are 13 unsorted sessions


In [4]:
for s in unsorted_sessions:
    if CLUSTER == 'Hoffman':
        cmd = f'labdata submit spksibl -a {SUBJECT} -s {s} --module matlab --conda-env spks --queue gpu,{GPU},h_rt=6:00:00,cuda=1 --delete-session --overwrite -- -t \$TMPDIR' # for hoffman
    elif CLUSTER == 'Hodgkin':
        cmd = f'labdata submit spksibl -a {SUBJECT} -s {s} -g 1 -n 8 -m 50 --delete-session --overwrite -- -t /scratch' # for hodgkin
    print(cmd)

labdata submit spksibl -a MM012 -s 2023-08-21/001 --module matlab --conda-env spks --queue gpu,RTX2080Ti,h_rt=6:00:00,cuda=1 --delete-session --overwrite -- -t \$TMPDIR
labdata submit spksibl -a MM012 -s 2023-08-24/001 --module matlab --conda-env spks --queue gpu,RTX2080Ti,h_rt=6:00:00,cuda=1 --delete-session --overwrite -- -t \$TMPDIR
labdata submit spksibl -a MM012 -s 2023-09-01/001 --module matlab --conda-env spks --queue gpu,RTX2080Ti,h_rt=6:00:00,cuda=1 --delete-session --overwrite -- -t \$TMPDIR
labdata submit spksibl -a MM012 -s 2023-09-07/001 --module matlab --conda-env spks --queue gpu,RTX2080Ti,h_rt=6:00:00,cuda=1 --delete-session --overwrite -- -t \$TMPDIR
labdata submit spksibl -a MM012 -s 2023-09-11/001 --module matlab --conda-env spks --queue gpu,RTX2080Ti,h_rt=6:00:00,cuda=1 --delete-session --overwrite -- -t \$TMPDIR
labdata submit spksibl -a MM012 -s 2023-09-12/001 --module matlab --conda-env spks --queue gpu,RTX2080Ti,h_rt=6:00:00,cuda=1 --delete-session --overwrite -